# Original

In [1]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from onlineTripletloss import *
from selector import *
from model_SAGAN1_1 import NetG, NetD, NetA
# from model_SAGAN2_Triplet import NetG, NetD, NetA
# from model_WGANGP import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_siGAN import NetG, NetD, NetA
# from data_set import CASIABDataset
from dataset2Loader_newtriplet import CASIABDataset
# from dataset2Loader_triplet import CASIABDataset
import torch.optim as optim
import visdom
from torchvision.utils import make_grid
# Data_Dir = '../GaitRecognition/DatasetB_GEI_64x64_allseq/'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_64x64_TripletSAGAN_90_trial9'
Model_dir = './Transform_Model/'+ Model_Name
if not os.path.isdir(Model_dir):
    os.mkdir(Model_dir)

In [ ]:

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
import os

for r, d, files in os.walk(Data_Dir):
    print(r)
    print(len(d))
    print(len(files))
    
# cpt = sum([len(files) for r, d, files in os.walk(Data_Dir)])
# cpt = sum([len(d) for r, d, files in os.walk(Data_Dir)])
# print(cpt)
# list = os.listdir(Data_Dir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# import fnmatch
# print(len(fnmatch.filter(os.listdir(Data_Dir), '*.png')))

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(16)

## iteration

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.xavier_normal_(tensor, gain=1.)
#         init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print('Training starts')
while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
    label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, label)
    lossD += lossD_real1.item()
    lossD_real1.backward()

    label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, label)
    lossD += lossD_real2.item()
    lossD_real2.backward()

    fake = netg(img).detach()
    label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, label)
    lossD += lossD_fake.item()
    lossD_fake.backward()

    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

    label.fill_(real_label)
    output1 = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output1, label)
    lossA += lossA_real1.item()
    lossA_real1.backward()

    label.fill_(fake_label)
    output = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output, label)
    lossA += lossA_real2.item()
    lossA_real2.backward()

    label.fill_(fake_label)
    output = neta(faked)
    lossA_fake = F.binary_cross_entropy(output, label)
    lossA += lossA_fake.item()
    lossA_fake.backward()
    
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)

    label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, label)
    lossG += lossGD.item()
    lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
    label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, label)
    lossG += lossGA.item()
    lossGA.backward()
    
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 25
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 1000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

    if iteration % 100==0 or iteration==10 :
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')
            
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))


## Epoch

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
train_loader = th.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=False)
train_loader.next()

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        



## update k times

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_img), 1)
            noassd = th.cat((img, noass_img), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))    
        

## model changed +Dp

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netp = NetP(nc=1)
neta = NetA(nc=1)
netd = NetD(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netp.children())[0].children(),
    list(neta.children())[0].children(),
    list(netd.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netp = netp.to(device)
neta = neta.to(device)
netd = netd.to(device)
netg.train()
netp.train()
neta.train()
netd.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimP = optim.Adam(netp.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        
        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()


        # update P
        lossP = 0
        optimP.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossP_real1 = F.binary_cross_entropy(output1, label)
        lossP += lossP_real1.item()/2
        lossP_real1 = lossP_real1/2
        lossP_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossP_real2 = F.binary_cross_entropy(output, label)
        lossP += lossP_real2.item()/2
        lossP_real2 = lossP_real2/2
        lossP_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossP_fake = F.binary_cross_entropy(output, label)
        lossP += lossP_fake.item()/2
        lossP_fake = lossP_fake/2
        lossP_fake.backward()

        optimP.step()
    

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        
        fake = netg(img)
        output = netd(fake)
        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward(retain_graph=True)
        
        faked = th.cat((img, fake), 1)
        output = netp(faked)
        label.fill_(real_label)
        lossGP = F.binary_cross_entropy(output, label)
        lossG += lossGP.item()/2
        lossGP = lossGP/2
        lossGP.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch, epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD', 'lossP']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch, epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}  \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        



## model only D_A

In [2]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
# netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
# optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
#             # update D
#             lossD = 0
#             optimD.zero_grad()
#             output = netd(ass_label)
#             label.fill_(real_label)
#             lossD_real1 = F.binary_cross_entropy(output, label)
#             lossD += lossD_real1.item()
#             lossD_real1.backward()

#             label.fill_(real_label)
#             output1 = netd(noass_label)
#             lossD_real2 = F.binary_cross_entropy(output1, label)
#             lossD += lossD_real2.item()
#             lossD_real2.backward()

#             fake = netg(img).detach()
#             label.fill_(fake_label)
#             output2 = netd(fake)
#             lossD_fake = F.binary_cross_entropy(output2, label)
#             lossD += lossD_fake.item()
#             lossD_fake.backward()

#             optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        ]]),
                           Y=np.array([[lossG/2, lossA/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  ]]),
                     Y=np.array([[lossG/2, lossA/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {} \n'.format(
            epoch, lossG/2, lossA/3,
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}'.format(
            epoch, lossG/2, lossA/3, 
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}\n'.format(
            epoch, lossG/2, lossA/3
        ))    
        

n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 0.8942790031433105, ErrA = 0.5469881097475687
Epoch = 4, ErrG = 1.2637784779071808, ErrA = 0.546068067351977
Epoch = 6, ErrG = 1.4745312631130219, ErrA = 0.2442850669225057
Epoch = 8, ErrG = 0.7030260860919952, ErrA = 0.5812898476918539
Epoch = 10, ErrG = 0.6319204270839691, ErrA = 0.5462305198113123
Epoch = 12, ErrG = 0.7229312062263489, ErrA = 0.5907560388247172
Epoch = 14, ErrG = 0.6560254096984863, ErrA = 0.5953078120946884
Epoch = 16, ErrG = 0.596514493227005, ErrA = 0.5941924850145975
Epoch = 18, ErrG = 0.7509839236736298, ErrA = 0.5790304243564606
Epoch = 20, ErrG = 0.7436962127685547, ErrA = 0.5637222131093343
Epoch = 22, ErrG = 0.7619479894638062, ErrA = 0.5524072647094727
Epoch = 24, ErrG = 0.7799927294254303, ErrA = 0.5675116032361984
Epoch = 26, ErrG = 0.7570993900299072, ErrA = 0.5430000573396683
Epoch = 28, ErrG = 0.6806648671627045, ErrA = 0.5911605507135391
Epoch = 30, ErrG = 0.7

Epoch = 250, ErrG = 0.6963056921958923, ErrA = 0.46742964078051347
Epoch = 252, ErrG = 0.7100014090538025, ErrA = 0.47177554422523826
Epoch = 254, ErrG = 0.7693454921245575, ErrA = 0.48319031205028296
Epoch = 256, ErrG = 0.7318943738937378, ErrA = 0.464450641341197
Epoch = 258, ErrG = 0.7589799463748932, ErrA = 0.4585082537184159
Epoch = 260, ErrG = 0.7342425584793091, ErrA = 0.46409560057024163
Epoch = 262, ErrG = 0.7421980202198029, ErrA = 0.4666421595029533
Epoch = 264, ErrG = 0.7296665608882904, ErrA = 0.48516935110092163
Epoch = 266, ErrG = 0.7399673163890839, ErrA = 0.4635310867646088
Epoch = 268, ErrG = 0.749874085187912, ErrA = 0.4334274607244879
Epoch = 270, ErrG = 0.7268602848052979, ErrA = 0.4545082812352727
Epoch = 272, ErrG = 0.7455301582813263, ErrA = 0.4670855162354807
Epoch = 274, ErrG = 0.7072040438652039, ErrA = 0.4976085623105367
Epoch = 276, ErrG = 0.7105807065963745, ErrA = 0.45866520221655566
Epoch = 278, ErrG = 0.7309572100639343, ErrA = 0.4670256379370888
Epoch 

Epoch = 498, ErrG = 0.8722212314605713, ErrA = 0.4581546305174318
Epoch = 500, ErrG = 0.7491079866886139, ErrA = 0.45177597298364464
Epoch = 502, ErrG = 0.7745728194713593, ErrA = 0.431646080124968
Epoch = 504, ErrG = 0.7889595925807953, ErrA = 0.46082764471066184
Epoch = 506, ErrG = 0.7366111278533936, ErrA = 0.46207964653149247
Epoch = 508, ErrG = 0.7318088412284851, ErrA = 0.35628407522259903
Epoch = 510, ErrG = 0.8470337688922882, ErrA = 0.3935772370386985
Epoch = 512, ErrG = 0.8028042912483215, ErrA = 0.45536067930515856
Epoch = 514, ErrG = 0.7802965939044952, ErrA = 0.47379907468954724
Epoch = 516, ErrG = 0.7577114105224609, ErrA = 0.4666667666460853
Epoch = 518, ErrG = 0.7378373742103577, ErrA = 0.458883986226283
Epoch = 520, ErrG = 0.777031272649765, ErrA = 0.45996388941421174
Epoch = 522, ErrG = 0.7456507086753845, ErrA = 0.46248103469163954
Epoch = 524, ErrG = 0.8133042454719543, ErrA = 0.46582059919213253
Epoch = 526, ErrG = 0.8049535155296326, ErrA = 0.46748004326218506
Epo

# Triplet 

In [ ]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from model_siGAN import NetG, NetD, NetA
from data_set import *
import torch.optim as optim
from torchvision.utils import make_grid

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(1)
# ass_label, noass_label, img = dataset.getbatch(1)

# dataset_test = CASIABDatasetForTest(data_dir=Data_Dir)
# ass_label_test, noass_label_test, img_test = dataset_test.getbatch(1)
# ass_label_test, noass_label_test, img_test = dataset_test.getbatch(1)

In [ ]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from model_siGAN import NetG, NetD, NetA
from data_set import *
import torch.optim as optim
from torchvision.utils import make_grid
import visdom
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_Triplet'
if not os.path.isdir(Model_Name):
    os.mkdir(Model_Name)
    
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
#     list(netd.children())[0].children(),
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netg.train()
dataset = CASIABDataset(data_dir=Data_Dir)
dataset_test = CASIABDatasetForTest(data_dir=Data_Dir)
ass_label_test, noass_label_test, img_test = dataset_test.getbatch(300)
ass_label_test = ass_label_test.to(device).to(th.float32)
noass_label_test = noass_label_test.to(device).to(th.float32)
img_test = img_test.to(device).to(th.float32)

iteration = 0
lr = 0.0001
batchSize = 128

optimG = optim.Adam(netg.parameters(), lr=lr)
# optimG = optim.Adam(netg.parameters(), lr=lr, weight_decay=1e-4)

while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    __, A = netg(img)
    __, P = netg(ass_label)
    __, N = netg(noass_label)
    lossTriplet = F.triplet_margin_loss(A, P, N, margin=5)
    lossGT = lossTriplet.item()
    lossTriplet.backward()
    optimG.step()
    
    iteration+=1
    if iteration % 500 == 0:
        with th.no_grad():
            netg.eval()  #切換
            __, A_test = netg(img_test)
            __, P_test = netg(ass_label_test)
            __, N_test = netg(noass_label_test)
            loss_test = F.triplet_margin_loss(A_test, P_test, N_test, margin=5)
            loss_test = loss_test.item()
            netg.train() #切換回去
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrGT = {}, Test_ErrGT = {}\n'.format(
            iteration, lossGT, loss_test
        ))
        print('iter = {}, ErrGT = {}, Test_ErrGT = {}'.format(
            iteration, lossGT, loss_test
        ))

        if win is None:
            win = vis.line(X=np.array([[iteration,iteration]]),
                           Y=np.array([[lossGT, loss_test]]),
                           opts=dict(
                               title= Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossGT', 'lossGT_test']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration]]),
                     Y=np.array([[lossGT, loss_test]]),
                     win=win,
                     update='append')
            
    if iteration % 2500 == 0:
        state = {
#             'netA': neta.state_dict(),
            'netG': netg.state_dict(),
#             'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

# Combine Siamese and GAN 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
margin = 5
target = '036'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, margin = {}, target={} \n'.format(
            epoches, lr, batchSize, margin, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake, _ = netg(img)
        label.fill_(fake_label)
        output2 = netd(fake.detach())
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()

        # update A
        lossA = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake,_ = netg(img)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked.detach())
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()
        
        
        # update G
        lossG = 0
        optimG.zero_grad()
        fake, A = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()
        
        # constrain on generator
        fake_ass, P = netg(ass_label)
        fake_noass, N = netg(noass_label)
        lossTriplet = F.triplet_margin_loss(fake, fake_ass, fake_noass, margin = margin)
        lossG += lossTriplet.item()
        lossTriplet.backward()
        
        # constrain on encoder
#         __, P = netg(ass_label)
#         __, N = netg(noass_label)
#         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
#         lossG += lossTriplet.item()
#         lossTriplet.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# Combine Siamese and GAN k times 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
margin = 10
g_k = 2
k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, margin = {}, g_k ={}, target={} \n'.format(
            epoches, lr, batchSize, margin, g_k, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
        
#         if(i ==0):
#             print(label_neg,label_anc,label_pos)
#             print(com_label)
#         print("shape",ass_label.shape,noass_label.shape,img.shape,com_img.shape, com_label.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach()) #需要 detach 因為不希望更新fake的參數
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#     if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake, A = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)  ##這裡需要retain graph 因為他之後有需要fake，因此需要retain

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()
        
        ## new tripletloss
        _, P = netg(ass_img)
        __, N = netg(noass_img)
        lossf = TripletLoss(margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
        lossTriplet =lossf(A, P, N)
        lossG += lossTriplet.item()
        lossTriplet.backward()
        
#         ## new onlinetripletloss
#         __, com = netg(com_img)
#         loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
#         lossTriplet,len_triplet = loss_fn(com, com_label)
#         lossG += lossTriplet.item()
#         lossTriplet.backward()
# #         print(lossTriplet.item(),len_triplet)
        
#         ## triplet loss
#         __, P = netg(ass_img)
#         __, N = netg(noass_img)
#         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
#         lossG += lossTriplet.item()
#         lossTriplet.backward()
# #         if i%10==0:
# #             print("tripletloss ",lossTriplet.item())

        ## tripletloss no negative
#         N_plus = th.zeros((A.size()), requires_grad=False).to(device)
#         lossTriplet_AP = F.triplet_margin_loss(A, P, N_plus, margin = margin)
#         lossG += lossTriplet_AP.item()
#         lossTriplet += lossTriplet_AP
#         lossTriplet.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG  Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# from PixelDT code

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

real_label = th.zeros((batchSize, 1), requires_grad=False).to(device)
fake_label = th.ones((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr)
optimD = optim.Adam(netd.parameters(), lr=lr)
optimA = optim.Adam(neta.parameters(), lr=lr)

print('Training starts')
while iteration < 40000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
#     label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, real_label)
#     lossD += lossD_real1.item()
#     lossD_real1.backward()

#     label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, real_label)
#     lossD += lossD_real2.item()
#     lossD_real2.backward()

    fake = netg(img).detach()
#     label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, fake_label)
#     lossD += lossD_fake.item()
#     lossD_fake.backward()
    lossD = (lossD_real1+ lossD_real2+ lossD_fake)/3
    lossD.backward()

    lossD_item = lossD.item()
    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

#     label.fill_(real_label)
    output = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output, real_label)
#     lossA += lossA_real1.item()
#     lossA_real1.backward()

#     label.fill_(fake_label)
    output1 = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output1, fake_label)
#     lossA += lossA_real2.item()
#     lossA_real2.backward()

#     label.fill_(fake_label)
    output2 = neta(faked)
    lossA_fake = F.binary_cross_entropy(output2, fake_label)
#     lossA += lossA_fake.item()
#     lossA_fake.backward()
    lossA = (lossA_real1+ lossA_real2 +lossA_fake)/3
    lossA.backward()
    
    lossA_item = lossA.item()
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)
#     label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, real_label)
#     lossG += lossGD.item()
#     lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
#     label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, real_label)
#     lossG += lossGA.item()
#     lossGA.backward()
    lossG = (lossGD + lossGA)/2
    lossG.backward()
    
    lossG_item = lossG.item()
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 5000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
    if iteration % 5000==0 or iteration==10 or iteration==500:
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                           opts=dict(
                               title='GaitGAN',
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                     win=win,
                     update='append')


# WGAN from gaitgan

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.RMSprop(netg.parameters(), lr=lr/2)
optimD = optim.RMSprop(netd.parameters(), lr=lr/3)
optimA = optim.RMSprop(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, n_critic = {}, target={} \n'.format(
            epoches, lr, batchSize, n_critic, target))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
#         label.fill_(real_label)
#         lossD_real1 = F.binary_cross_entropy(output, label)
        lossD_real1 = -th.mean(output)
        lossD += lossD_real1.item()
        lossD_real1.backward()

#         label.fill_(real_label)
        output1 = netd(noass_label)
#         lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD_real2 = -th.mean(output1)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
#         label.fill_(fake_label)
        output2 = netd(fake)
#         lossD_fake = F.binary_cross_entropy(output2, label)
        lossD_fake = th.mean(output2)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()
        
        for p in netd.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake = netg(img).detach()
        faked = th.cat((img, fake), 1)

#         label.fill_(real_label)
        output1 = neta(assd)
#         lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA_real1 = -th.mean(output1)
        lossA += lossA_real1.item()
        lossA_real1.backward()

#         label.fill_(fake_label)
        output = neta(noassd)
#         lossA_real2 = F.binary_cross_entropy(output, label)
        lossA_real2 = th.mean(output)
        lossA += lossA_real2.item()
        lossA_real2.backward()

#         label.fill_(fake_label)
        output = neta(faked)
#         lossA_fake = F.binary_cross_entropy(output, label)
        lossA_fake = th.mean(output)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()
    
        for p in neta.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        # update G
        if i % n_critic == 0:
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
        
        


 # WGAN-GP

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 5
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.999

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=lr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=lr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=lr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, lr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        output = netd(ass_label)
#         label.fill_(real_label)
#         lossD_real1 = F.binary_cross_entropy(output, label)
        lossD_real1 = -th.mean(output)
        lossD_ += lossD_real1
        lossD += lossD_real1.item()
#         lossD_real1.backward()

#         label.fill_(real_label)
        output1 = netd(noass_label)
#         lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD_real2 = -th.mean(output1)
        lossD_ += lossD_real2
        lossD += lossD_real2.item()
#         lossD_real2.backward()

        fake = netg(img).detach()
#         label.fill_(fake_label)
        output2 = netd(fake)
#         lossD_fake = F.binary_cross_entropy(output2, label)
        lossD_fake = th.mean(output2)
        lossD_ += lossD_fake
        lossD += lossD_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
#         lossD_fake.backward()
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
        lossD_.backward()

        optimD.step()
        
#         for p in netd.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        fake = netg(img).detach()
        faked = th.cat((img, fake), 1)

#         label.fill_(real_label)
        output1 = neta(assd)
#         lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA_real1 = -th.mean(output1)
        lossA += lossA_real1.item()
        lossA_ += lossA_real1
#         lossA_real1.backward()

#         label.fill_(fake_label)
        output = neta(noassd)
#         lossA_real2 = F.binary_cross_entropy(output, label)
        lossA_real2 = th.mean(output)
        lossA += lossA_real2.item()
        lossA_ += lossA_real2
#         lossA_real2.backward()

#         label.fill_(fake_label)
        output = neta(faked)
#         lossA_fake = F.binary_cross_entropy(output, label)
        lossA_fake = th.mean(output)
        lossA += lossA_fake.item()
        lossA_ += lossA_fake
#         lossA_fake.backward()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
        
        lossA_.backward()
        optimA.step()
    
#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
        if i % n_critic == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossG_ += lossGD
#             lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossG_ += lossGA
#             lossGA.backward()
            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        
        


# SA GaitGAN (hing)

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
#         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
#         for p in netd.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
#         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()
    
#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# SA GaitGAN (WGAN-GP)

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        
        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = -th.mean(d_out_assreal)

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = -th.mean(d_out_noassreal)

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = th.mean(d_out_fake)

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
#         lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = -th.mean(d_out_assreal)
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = th.mean(d_out_noassreal)

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = th.mean(d_out_faked)

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
#         lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()

            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# combine siamese and SAGAN 

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
g_k = 2

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, g_k={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, g_k, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % g_k ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()

    #         for p in netd.parameters():
    #             p.data.clamp_(-0.01, 0.01)

            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake, A= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        # constrain on generator
        fake_ass, P = netg(ass_img)
        fake_noass, N = netg(noass_img)
        lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
        lossG_ += lossTriplet
        lossG += lossTriplet.item()
#         lossTriplet.backward()
        
        lossG_ = lossG_/3
        lossG_.backward(retain_graph=True)
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 1.9260542591412861, ErrTri = 5.951812744140625, ErrA = 0.676393223926425, ErrD = 0.6965182224909464, Gattn=-0.003843697952106595, Dattn=0.0010151999304071069, Aattn=4.331633681431413e-05
Epoch = 4, ErrG = 1.6584657430648804, ErrTri = 5.226146221160889, ErrA = 0.6802028510719538, ErrD = 0.6874451587597529, Gattn=-0.005840306170284748, Dattn=0.0007879249169491231, Aattn=5.519977639778517e-05
Epoch = 6, ErrG = 1.7611311674118042, ErrTri = 5.390015125274658, ErrA = 0.6771195195615292, ErrD = 0.7037542636195818, Gattn=-0.0068143089301884174, Dattn=0.0010411073453724384, Aattn=2.1014287995058112e-05
Epoch = 8, ErrG = 1.2754840453465779, ErrTri = 3.8532814979553223, ErrA = 0.6971508264541626, ErrD = 0.698284812271595, Gattn=-0.007755768019706011, Dattn=0.002113168127834797, Aattn=-3.813387593254447e-05
Epoch = 10, E

Epoch = 80, ErrG = 0.35619552930196124, ErrTri = 1.1775492429733276, ErrA = 0.6695427081237236, ErrD = 0.6721634231507778, Gattn=-0.01801953837275505, Dattn=0.037466391921043396, Aattn=-0.006200551055371761
Epoch = 82, ErrG = 0.6228386163711548, ErrTri = 1.893239974975586, ErrA = 0.6658571660518646, ErrD = 0.6740605961531401, Gattn=-0.018170703202486038, Dattn=0.0378650426864624, Aattn=-0.006782026030123234
Epoch = 84, ErrG = 0.7426910797754923, ErrTri = 2.2517428398132324, ErrA = 0.66833905937771, ErrD = 0.6767601855099201, Gattn=-0.01821422390639782, Dattn=0.03823858126997948, Aattn=-0.007006083615124226
Epoch = 86, ErrG = 0.6266598502794901, ErrTri = 1.9032856225967407, ErrA = 0.668173398822546, ErrD = 0.6745888255536556, Gattn=-0.018352419137954712, Dattn=0.03859074413776398, Aattn=-0.007284077815711498
Epoch = 88, ErrG = 0.40512855847676593, ErrTri = 1.3287544250488281, ErrA = 0.6693551825980345, ErrD = 0.6740432791411877, Gattn=-0.01846087910234928, Dattn=0.03890705108642578, Aat

Epoch = 160, ErrG = 0.26708030700683594, ErrTri = 0.8404383659362793, ErrA = 0.6729234041025242, ErrD = 0.6662080648044745, Gattn=-0.02051149122416973, Dattn=0.04478449001908302, Aattn=-0.025810878723859787
Epoch = 162, ErrG = 0.2933942874272664, ErrTri = 0.8757205009460449, ErrA = 0.6733941199878851, ErrD = 0.6670015857865413, Gattn=-0.020526675507426262, Dattn=0.04490313678979874, Aattn=-0.026120126247406006
Epoch = 164, ErrG = 0.5324180126190186, ErrTri = 1.6377794742584229, ErrA = 0.6684541472544273, ErrD = 0.6698337029665709, Gattn=-0.020552387461066246, Dattn=0.04499809816479683, Aattn=-0.026392705738544464
Epoch = 166, ErrG = 0.3370840549468994, ErrTri = 1.0034589767456055, ErrA = 0.6620852078000704, ErrD = 0.6629095959166685, Gattn=-0.020608749240636826, Dattn=0.04509630799293518, Aattn=-0.026749679818749428
Epoch = 168, ErrG = 0.3410554528236389, ErrTri = 1.0329116582870483, ErrA = 0.6664159862945477, ErrD = 0.6682561058551073, Gattn=-0.020655827596783638, Dattn=0.045173820108

Epoch = 240, ErrG = 0.21528776486714682, ErrTri = 0.6162649393081665, ErrA = 0.6447196900844574, ErrD = 0.6581355830033621, Gattn=-0.02223062328994274, Dattn=0.0522821806371212, Aattn=-0.048486292362213135
Epoch = 242, ErrG = 0.20778397719065347, ErrTri = 0.5268774628639221, ErrA = 0.6685782615095377, ErrD = 0.6646854567031065, Gattn=-0.022338805720210075, Dattn=0.05235536769032478, Aattn=-0.04984793812036514
Epoch = 244, ErrG = 0.22306394577026367, ErrTri = 0.60271817445755, ErrA = 0.6618240413566431, ErrD = 0.6601707606265942, Gattn=-0.022450676187872887, Dattn=0.052425190806388855, Aattn=-0.04975678026676178
Epoch = 246, ErrG = 0.23902557293574014, ErrTri = 0.6476641297340393, ErrA = 0.6654705386608839, ErrD = 0.6563515824576219, Gattn=-0.022485367953777313, Dattn=0.0525280199944973, Aattn=-0.05056845396757126
Epoch = 248, ErrG = 0.1009166141351064, ErrTri = 0.2766213119029999, ErrA = 0.6410699114203453, ErrD = 0.6393701086441675, Gattn=-0.022413745522499084, Dattn=0.052645143121480

Epoch = 320, ErrG = 0.3705063462257385, ErrTri = 0.6936146020889282, ErrA = 0.592785914738973, ErrD = 0.7291649350275596, Gattn=-0.02384408749639988, Dattn=0.04530612751841545, Aattn=-0.07142718881368637
Epoch = 322, ErrG = 0.36221540967623395, ErrTri = 0.41553181409835815, ErrA = 0.6385101415216923, ErrD = 0.6386140187581381, Gattn=-0.023841699585318565, Dattn=0.04530642554163933, Aattn=-0.07174340635538101
Epoch = 324, ErrG = 0.4301662743091583, ErrTri = 0.5181174874305725, ErrA = 0.5364030400911967, ErrD = 0.6731035814930996, Gattn=-0.023899540305137634, Dattn=0.04536827281117439, Aattn=-0.07195452600717545
Epoch = 326, ErrG = 0.7021071414152781, ErrTri = 0.4537392556667328, ErrA = 0.5342137515544891, ErrD = 0.4386960168679555, Gattn=-0.023977473378181458, Dattn=0.04535771906375885, Aattn=-0.07231582701206207
Epoch = 328, ErrG = 0.14137120048205057, ErrTri = 0.3223157823085785, ErrA = 0.6430610101670027, ErrD = 0.5159811973571777, Gattn=-0.02394130639731884, Dattn=0.0453781746327877

Epoch = 400, ErrG = 0.9914257526397705, ErrTri = 0.451373815536499, ErrA = 0.4774714633822441, ErrD = 0.5560151661435763, Gattn=-0.025720078498125076, Dattn=0.04532025381922722, Aattn=-0.08039143681526184
Epoch = 402, ErrG = 0.9281780322392782, ErrTri = 0.49386441707611084, ErrA = 0.575357586145401, ErrD = 0.4404910746961832, Gattn=-0.025699876248836517, Dattn=0.04528258368372917, Aattn=-0.0808243378996849
Epoch = 404, ErrG = 0.5898028910160065, ErrTri = 0.5013291239738464, ErrA = 0.5079388494292895, ErrD = 0.5758944184829792, Gattn=-0.0258130244910717, Dattn=0.04514315724372864, Aattn=-0.08118374645709991
Epoch = 406, ErrG = 0.956541121006012, ErrTri = 0.28301918506622314, ErrA = 0.4219586451848348, ErrD = 0.4264504512151082, Gattn=-0.025800831615924835, Dattn=0.04506335034966469, Aattn=-0.08156149834394455
Epoch = 408, ErrG = 0.5613180100917816, ErrTri = 0.3418649137020111, ErrA = 0.38512905811270076, ErrD = 0.5154022177060446, Gattn=-0.025920584797859192, Dattn=0.04513439163565636, 

Epoch = 480, ErrG = 0.7129133393367132, ErrTri = 1.0005292892456055, ErrA = 0.31649766365687054, ErrD = 0.3707296848297119, Gattn=-0.027231695130467415, Dattn=0.04338693991303444, Aattn=-0.08908867835998535
Epoch = 482, ErrG = 0.7922591716051102, ErrTri = 0.23373650014400482, ErrA = 0.33417196571826935, ErrD = 0.3600047454237938, Gattn=-0.027247117832303047, Dattn=0.04333580657839775, Aattn=-0.08913403749465942
Epoch = 484, ErrG = 0.8346841633319855, ErrTri = 0.38902977108955383, ErrA = 0.30527279898524284, ErrD = 0.36581435054540634, Gattn=-0.027277855202555656, Dattn=0.043313074856996536, Aattn=-0.08927620202302933
Epoch = 486, ErrG = 0.5601225395997366, ErrTri = 0.32630059123039246, ErrA = 0.3070090711116791, ErrD = 0.24749239285786948, Gattn=-0.027351893484592438, Dattn=0.043239135295152664, Aattn=-0.08956865966320038
Epoch = 488, ErrG = 0.2496929963429769, ErrTri = 0.26613765954971313, ErrA = 0.38843360481162864, ErrD = 0.48542119065920514, Gattn=-0.027309393510222435, Dattn=0.043

Epoch = 560, ErrG = 0.6118245323499044, ErrTri = 0.4160063564777374, ErrA = 0.25557664781808853, ErrD = 0.39010149488846463, Gattn=-0.028949540108442307, Dattn=0.04092749208211899, Aattn=-0.0962422788143158
Epoch = 562, ErrG = 0.46231236557165784, ErrTri = 0.3709062337875366, ErrA = 0.1954436351855596, ErrD = 0.4338449239730835, Gattn=-0.0289840679615736, Dattn=0.040781352669000626, Aattn=-0.0964789167046547
Epoch = 564, ErrG = 0.6003871162732443, ErrTri = 0.5088762044906616, ErrA = 0.23497148354848227, ErrD = 0.403876930475235, Gattn=-0.02902291715145111, Dattn=0.04073943942785263, Aattn=-0.09673493355512619
Epoch = 566, ErrG = 0.38870836421847343, ErrTri = 0.2965693473815918, ErrA = 0.13640687863032022, ErrD = 0.4274595081806183, Gattn=-0.029039878398180008, Dattn=0.04067516326904297, Aattn=-0.09677167236804962
Epoch = 568, ErrG = 0.8463039000829061, ErrTri = 0.40488290786743164, ErrA = 0.20413067440191904, ErrD = 0.28954119980335236, Gattn=-0.02909497357904911, Dattn=0.0405994541943

Epoch = 640, ErrG = 0.5660357077916464, ErrTri = 0.2698623538017273, ErrA = 0.2882832909623782, ErrD = 0.4440277914206187, Gattn=-0.03037169761955738, Dattn=0.037870854139328, Aattn=-0.10285911709070206
Epoch = 642, ErrG = 1.012982447942098, ErrTri = 0.4905036687850952, ErrA = 0.19511589407920837, ErrD = 0.3192889454464118, Gattn=-0.030424263328313828, Dattn=0.03771147131919861, Aattn=-0.10301701724529266
Epoch = 644, ErrG = 0.44807885338862735, ErrTri = 0.3806682825088501, ErrA = 0.1994085634748141, ErrD = 0.2497753451267878, Gattn=-0.03044893778860569, Dattn=0.03765394166111946, Aattn=-0.10325406491756439
Epoch = 646, ErrG = 0.5317885875701904, ErrTri = 0.4957934021949768, ErrA = 0.13783507173260054, ErrD = 0.5832590957482656, Gattn=-0.03045841120183468, Dattn=0.03764573484659195, Aattn=-0.10344134271144867
Epoch = 648, ErrG = 0.274221807718277, ErrTri = 0.3355322480201721, ErrA = 0.2705098930746317, ErrD = 0.4238609845439593, Gattn=-0.03050369955599308, Dattn=0.0375526137650013, Aat

# SA and triplet (onlineloss)

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:0")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
g_k = 2

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, g_k={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, g_k, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img, label_neg, label_anc, label_pos) in enumerate(train_loader):
        
        com_img = th.cat((noass_img, img, ass_img), 0)
        com_label = th.cat(( label_neg, label_anc, label_pos), 0)
        com_img = com_img.to(device).to(th.float32)
        com_label = com_label.to(device).to(th.float32)
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % g_k ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()

    #         for p in netd.parameters():
    #             p.data.clamp_(-0.01, 0.01)

            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

#         for p in neta.parameters():
#             p.data.clamp_(-0.01, 0.01)
            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake, A= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
#         # constrain on encoder
#         fake_ass, P = netg(ass_img)
#         fake_noass, N = netg(noass_img)
#         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
#         lossG_ += lossTriplet
#         lossG += lossTriplet.item()
# #         lossTriplet.backward()

        ## new onlinetripletloss
        __, com = netg(com_img)
        loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
        lossTriplet,len_triplet = loss_fn(com, com_label)
        lossG += lossTriplet.item()
        lossG_ += lossTriplet
#         lossTriplet.backward()
        
        
        lossG_ = lossG_/3
        lossG_.backward(retain_graph=True)
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 4.3307035366694135, ErrTri = 10.077913284301758, ErrA = 0.5983563270419836, ErrD = 0.0014277820785840352, Gattn=-0.004065600223839283, Dattn=0.0015907030319795012, Aattn=-0.0004191520274616778
Epoch = 4, ErrG = 4.570343852043152, ErrTri = 9.814889907836914, ErrA = 0.587698737780253, ErrD = 0.0, Gattn=-0.006797180976718664, Dattn=0.0015751610044389963, Aattn=-0.0011272769188508391
Epoch = 6, ErrG = 4.4995341300964355, ErrTri = 9.811159133911133, ErrA = 0.4260034207254648, ErrD = 0.0, Gattn=-0.008517315611243248, Dattn=0.0017938892124220729, Aattn=-0.0013575409539043903
Epoch = 8, ErrG = 4.231283028920491, ErrTri = 9.802407264709473, ErrA = 0.3957052454352379, ErrD = 0.01120259923239549, Gattn=-0.009669558145105839, Dattn=0.0026833228766918182, Aattn=-0.0018445397727191448
Epoch = 10, ErrG = 4.581889986991882, 

Epoch = 82, ErrG = 2.68929123878479, ErrTri = 5.80018424987793, ErrA = 0.12432180407146613, ErrD = 0.028253187735875446, Gattn=-0.014779558405280113, Dattn=0.038581665605306625, Aattn=-0.05178460106253624
Epoch = 84, ErrG = 3.8530918757120767, ErrTri = 9.642534255981445, ErrA = 0.23682251324256262, ErrD = 0.27374399701754254, Gattn=-0.014904660172760487, Dattn=0.03903748467564583, Aattn=-0.05211389437317848
Epoch = 86, ErrG = 4.115328272183736, ErrTri = 9.569890975952148, ErrA = 0.2966725528240204, ErrD = 0.02949485679467519, Gattn=-0.014925678260624409, Dattn=0.03950992971658707, Aattn=-0.052520427852869034
Epoch = 88, ErrG = 4.1760051647822065, ErrTri = 9.699357032775879, ErrA = 0.14357932905356088, ErrD = 0.6229323546091715, Gattn=-0.015039564110338688, Dattn=0.03992605209350586, Aattn=-0.05310096591711044
Epoch = 90, ErrG = 3.9172569115956626, ErrTri = 9.38385009765625, ErrA = 0.36640518965820473, ErrD = 0.008731112504998842, Gattn=-0.01512134075164795, Dattn=0.04062696918845177, A

Epoch = 162, ErrG = 4.128449755410354, ErrTri = 9.99888801574707, ErrA = 0.343599076072375, ErrD = 0.1928730315218369, Gattn=-0.01871705800294876, Dattn=0.061338502913713455, Aattn=-0.07558640092611313
Epoch = 164, ErrG = 1.7012173334757488, ErrTri = 3.1829614639282227, ErrA = 0.22004913787047067, ErrD = 0.12656701356172562, Gattn=-0.018728982657194138, Dattn=0.061885859817266464, Aattn=-0.07621489465236664
Epoch = 166, ErrG = 1.943870057662328, ErrTri = 4.487652778625488, ErrA = 0.3034538726011912, ErrD = 0.1578477919101715, Gattn=-0.018761886283755302, Dattn=0.06244196742773056, Aattn=-0.07718972861766815
Epoch = 168, ErrG = 3.9903079668680825, ErrTri = 9.616950035095215, ErrA = 0.3362627327442169, ErrD = 0.33472386995951336, Gattn=-0.01877123862504959, Dattn=0.06288199126720428, Aattn=-0.0780491828918457
Epoch = 170, ErrG = 3.4994009335835776, ErrTri = 10.16681957244873, ErrA = 0.3931317826112111, ErrD = 0.43600266178448993, Gattn=-0.01880337856709957, Dattn=0.06359629333019257, Aat

Epoch = 244, ErrG = 1.0071096817652385, ErrTri = 1.084277868270874, ErrA = 0.32825985302527744, ErrD = 0.11804355743030708, Gattn=-0.019343221560120583, Dattn=0.0860864669084549, Aattn=-0.09743905812501907
Epoch = 246, ErrG = 3.6193078060944877, ErrTri = 9.816554069519043, ErrA = 0.2790232114493847, ErrD = 0.25952573865652084, Gattn=-0.019336150959134102, Dattn=0.0870877206325531, Aattn=-0.0980335995554924
Epoch = 248, ErrG = 1.0319848358631134, ErrTri = 0.2466789186000824, ErrA = 0.25844497481981915, ErrD = 0.04624846950173378, Gattn=-0.019349075853824615, Dattn=0.0882432833313942, Aattn=-0.09882457554340363
Epoch = 250, ErrG = 3.9609712958335876, ErrTri = 10.42411994934082, ErrA = 0.3388238816211621, ErrD = 0.46070006862282753, Gattn=-0.019371023401618004, Dattn=0.0889633446931839, Aattn=-0.09939240664243698
Epoch = 252, ErrG = 0.8702847063541412, ErrTri = 0.3744780719280243, ErrA = 0.43620812768737477, ErrD = 0.06906209886074066, Gattn=-0.019433818757534027, Dattn=0.0906716287136077

Epoch = 324, ErrG = 5.155336618423462, ErrTri = 12.755003929138184, ErrA = 0.35022486311693984, ErrD = 0.11016619205474854, Gattn=-0.020653896033763885, Dattn=0.12332236766815186, Aattn=-0.1382359266281128
Epoch = 326, ErrG = 4.190668185551961, ErrTri = 10.473072052001953, ErrA = 0.39090534547964734, ErrD = 0.2281729852159818, Gattn=-0.020697062835097313, Dattn=0.12417209893465042, Aattn=-0.14053158462047577
Epoch = 328, ErrG = 1.2437903881072998, ErrTri = 1.8172876834869385, ErrA = 0.42406286609669525, ErrD = 0.08127555375297864, Gattn=-0.020720522850751877, Dattn=0.12540671229362488, Aattn=-0.1428118646144867
Epoch = 330, ErrG = 4.240328788757324, ErrTri = 9.743684768676758, ErrA = 0.42521550754706067, ErrD = 0.22595086445411047, Gattn=-0.020757777616381645, Dattn=0.12609520554542542, Aattn=-0.1448320597410202
Epoch = 332, ErrG = 0.1500114103158315, ErrTri = 0.1930597722530365, ErrA = 0.4646188039332628, ErrD = 0.21253055334091187, Gattn=-0.020817402750253677, Dattn=0.126994952559471

Epoch = 406, ErrG = 0.7571269969145457, ErrTri = 0.29306191205978394, ErrA = 0.4203989952802658, ErrD = 0.19616460800170898, Gattn=-0.020808937028050423, Dattn=0.16738475859165192, Aattn=-0.1925671249628067
Epoch = 408, ErrG = 1.0465591351191204, ErrTri = 0.0, ErrA = 0.37700246398647624, ErrD = 0.08527581642071407, Gattn=-0.020805012434720993, Dattn=0.16780418157577515, Aattn=-0.19352415204048157
Epoch = 410, ErrG = 1.0450934966405232, ErrTri = 0.0, ErrA = 0.326434791708986, ErrD = 0.049354170759518944, Gattn=-0.020829861983656883, Dattn=0.1692594289779663, Aattn=-0.19467999041080475
Epoch = 412, ErrG = 1.9836668968200684, ErrTri = 3.778325080871582, ErrA = 0.378379558523496, ErrD = 0.2872329205274582, Gattn=-0.020785728469491005, Dattn=0.17066816985607147, Aattn=-0.19549936056137085
Epoch = 414, ErrG = 1.0617190996805828, ErrTri = 1.0440423488616943, ErrA = 0.3486352364222209, ErrD = 0.08083800971508026, Gattn=-0.02079119347035885, Dattn=0.17230217158794403, Aattn=-0.19694340229034424

Epoch = 490, ErrG = 4.7020905415217085, ErrTri = 10.823177337646484, ErrA = 0.3771978449076414, ErrD = 0.03912800674637159, Gattn=-0.020678821951150894, Dattn=0.21006715297698975, Aattn=-0.23239877820014954
Epoch = 492, ErrG = 0.9579919974009196, ErrTri = 0.2175493836402893, ErrA = 0.27708712220191956, ErrD = 0.014244688053925833, Gattn=-0.020683683454990387, Dattn=0.21102608740329742, Aattn=-0.2335713654756546
Epoch = 494, ErrG = 0.9906209309895834, ErrTri = 0.0, ErrA = 0.3367416262626648, ErrD = 0.0817548433939616, Gattn=-0.02068457379937172, Dattn=0.21208897233009338, Aattn=-0.23451362550258636
Epoch = 496, ErrG = 1.0457416772842407, ErrTri = 0.0, ErrA = 0.2506762941678365, ErrD = 0.03459858149290085, Gattn=-0.02068120799958706, Dattn=0.21304908394813538, Aattn=-0.23544807732105255
Epoch = 498, ErrG = 1.1669177214304607, ErrTri = 0.0, ErrA = 0.17886242270469666, ErrD = 0.06544124459226926, Gattn=-0.020670903846621513, Dattn=0.2144005000591278, Aattn=-0.23582319915294647
Epoch = 500,

Epoch = 574, ErrG = 1.0815411011377971, ErrTri = 0.0, ErrA = 0.277199645837148, ErrD = 0.08733576163649559, Gattn=-0.02057822048664093, Dattn=0.24921683967113495, Aattn=-0.260517954826355
Epoch = 576, ErrG = 0.8394650419553121, ErrTri = 0.0, ErrA = 0.292604794104894, ErrD = 0.05209909689923128, Gattn=-0.020581144839525223, Dattn=0.2498510330915451, Aattn=-0.261298269033432
Epoch = 578, ErrG = 4.851070165634155, ErrTri = 12.08614444732666, ErrA = 0.17063714812199274, ErrD = 0.06521530449390411, Gattn=-0.020595228299498558, Dattn=0.2506290674209595, Aattn=-0.2618115544319153
Epoch = 580, ErrG = 0.9761586586634318, ErrTri = 0.20535218715667725, ErrA = 0.19876286884148917, ErrD = 0.11135278766353925, Gattn=-0.020594267174601555, Dattn=0.2510722279548645, Aattn=-0.26209700107574463
Epoch = 582, ErrG = 2.2063461542129517, ErrTri = 3.735238552093506, ErrA = 0.24119327465693155, ErrD = 0.1022436333199342, Gattn=-0.020585210993885994, Dattn=0.25163453817367554, Aattn=-0.2630246877670288
Epoch =

Epoch = 658, ErrG = 1.2122105558713276, ErrTri = 0.18145626783370972, ErrA = 0.18896597623825073, ErrD = 0.12144210934638977, Gattn=-0.020482417196035385, Dattn=0.2770558297634125, Aattn=-0.27848631143569946
Epoch = 660, ErrG = 0.935682455698649, ErrTri = 0.0, ErrA = 0.2325686551630497, ErrD = 0.12889100362857184, Gattn=-0.020486675202846527, Dattn=0.2778032124042511, Aattn=-0.2788603603839874
Epoch = 662, ErrG = 0.9040336211522421, ErrTri = 0.0, ErrA = 0.09610960943003495, ErrD = 0.04918175501128038, Gattn=-0.020493902266025543, Dattn=0.2788173258304596, Aattn=-0.27942603826522827
Epoch = 664, ErrG = 1.297120988368988, ErrTri = 0.2180578112602234, ErrA = 0.14010012708604336, ErrD = 0.05550375704964002, Gattn=-0.02049519121646881, Dattn=0.27889394760131836, Aattn=-0.27997803688049316
Epoch = 666, ErrG = 0.8815619150797526, ErrTri = 0.0, ErrA = 0.16299923757712045, ErrD = 0.11556508267919223, Gattn=-0.02048136293888092, Dattn=0.27959147095680237, Aattn=-0.28002581000328064
Epoch = 668, 